# Cтруктура аудитории и доля
Пример расчета соц-дем структуры аудитории телеканалов.

## Описание задачи и условий расчета
- Период: 01.08.2022 - 31.08.2022
- Временной интервал: 05:00-29:00
- ЦА: Все 4+ в разибвке по полу, образованию, занятости, возрастным группам и роду занятий
- Место просмотра: Все места (дом+дача)
- Каналы: ПЕРВЫЙ КАНАЛ, РОССИЯ 1, НТВ									
- Статистики: Share, Rtg000, Rtg%

## Инициализация

При построении отчета первый шаг в любом ноутбуке - загрузка библиотек, которые помогут обращаться к TVI API и работать с данными.

Выполните следующую ячейку, для этого перейдите в нее и нажмите Ctrl+Enter

In [1]:
%reload_ext autoreload
%autoreload 2

import sys
import os
import re
import json
import datetime
import time
import pandas as pd
from IPython.display import JSON

from mediascope_api.core import net as mscore
from mediascope_api.mediavortex import tasks as cwt
from mediascope_api.mediavortex import catalogs as cwc

# Настраиваем отображение

# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)
# Задаем максимальное количество выводимых строк. Раскомментируйте нужную строку
# 200 строк
# pd.set_option("display.max_rows", 200)
# Отображаем все строки. ВАЖНО! Отображение большого DataFrame требует много ресурсов
# pd.set_option("display.max_rows", None)

# Cоздаем объекты для работы с TVI API
mnet = mscore.MediascopeApiNetwork()
mtask = cwt.MediaVortexTask()
cats = cwc.MediaVortexCats()

Запрошены записи: 0 - 257
Всего найдено записей: 257

Запрошены записи: 0 - 257
Всего найдено записей: 257



## Справочники

In [2]:
# выберем телесети по имени (части имени)
cats.get_tv_net(name=["ПЕРВЫЙ КАНАЛ", "РОССИЯ 1", "НТВ"])

# далее в запросах будем использовать id телесетей.

Запрошены записи: 0 - 0
Всего найдено записей: 0



""


In [3]:
# Найдем названия нужных нам демографических переменных
cats.get_tv_demo_attribute(names=['пол','образование','занятость','возрастная группа (wgh)','род занятий'])

Запрошены записи: 0 - 30
Всего найдено записей: 30



,id,valueId,name,colName,valueName
0,1,1,Пол,Sex,Мужчины
1,1,2,Пол,Sex,Женщины
2,3,1,Образование,Education,начальное
3,3,2,Образование,Education,среднее
4,3,3,Образование,Education,высшее
5,3,99,Образование,Education,затрудняюсь ответить (з/о)
6,26,1,Семейное положение,MaritalStatus,Женат / замужем
7,26,2,Семейное положение,MaritalStatus,Холост / не замужем
8,22,10,Род занятий,Status,"Владелец, совладелец предприятия"
9,22,2,Род занятий,Status,Специалист


## Формирование задания
Зададим условия расчета

In [4]:
# Период указывается в виде списка ('Начало', 'Конец'). 
date_filter = [('2022-08-01', '2022-08-31')]

# Задаем дни недели
weekday_filter = None

# Задаем тип дня
daytype_filter = None

# Задаем временной интервал
time_filter = None

# Задаем ЦА
basedemo_filter = None

# Доп фильтр ЦА, нужен только в случае расчета отношения между ЦА, например, при расчете Affinity Index
targetdemo_filter = None

# Задаем место просмотра
location_filter = None

# Задаем каналы
company_filter = 'tvNetId IN (1, 2, 4)'

# Указываем список статистик для расчета
statistics = ['Share', 'Rtg000', 'RtgPer']

# Указываем срезы
slices = ['tvNetName']

# Задаем опции расчета
options = {
    "kitId": 1, #TV Index Russia all 
}

Формируем различные срезы по демографии

In [5]:
# Создадим словарь с демо атрибутами, где ключ - name, а значение entityName атрибута  
demos = {'Пол':'sex', 
             'Образование':'education', 
             'Занятость':'work', 
             'Возрастная группа':'wghSuburbAgeGroup', 
             'Род занятий':'status'}

## Расчет задания

In [6]:
# Посчитаем задания в цикле
results = []
print("Отправляем задания на расчет")

# Каждый элемент списка подставляется в параметр slices
for key, value in demos.items():
    project_name = key
    slices.append(value)

    # Формируем задание для TV API в формате JSON
    task_json = mtask.build_task('timeband', task_name=project_name, date_filter=date_filter, 
                                 weekday_filter=weekday_filter, daytype_filter=daytype_filter, 
                                 company_filter=company_filter, time_filter=time_filter, 
                                 basedemo_filter=basedemo_filter, targetdemo_filter=targetdemo_filter,
                                 location_filter=location_filter, slices=slices, 
                                 statistics=statistics, options=options)

    # Отправляем задание на расчет и ждем выполнения
    task_timeband = mtask.wait_task(mtask.send_timeband_task(task_json))

    # Получаем результат
    df_timeband = mtask.result2table(mtask.get_result(task_timeband), project_name=project_name)
    
    # Меняем название колонки с демо атрибутом для последующего объединения таблиц
    df_timeband.rename(columns={value: 'valueName'}, inplace=True)
    
    #Сохраняем таблицу
    results.append(df_timeband)
    
    #Удаляем демо атрибут из разбивки
    slices.remove(value)

# Объединяем все полученные таблицы в одну
df = pd.concat(results)
df    

Отправляем задания на расчет
Расчет задачи (id: 7603103f-fba6-4352-a692-84eb4adfe2eb) [= = = = = = = = = = = = = = = = = = = = = = = = ] время расчета: 0:01:18.662204
Расчет задачи (id: d1d2fcab-7376-4ae6-b646-5427dcbc7ccb) [= = = = = = = = = = = = = = = = = = = = = = ] время расчета: 0:01:11.275686
Расчет задачи (id: 0eadcc5a-39d8-40d3-918a-36e7bf89724f) [= = = = = = = = = = = = = = = = = = = = ] время расчета: 0:01:04.410314
Расчет задачи (id: 1c07d976-3545-4781-bba6-35606bd71456) [= = = = = = = = = = = = = = = = = = = = ] время расчета: 0:01:04.801410
Расчет задачи (id: 887575e5-0501-4e0c-af73-a08411194c35) [= = = = = = = = = = = = = = = = = = = = = ] время расчета: 0:01:07.843711


,prj_name,valueName,tvNetName,Share,RtgPer,Rtg000
0,Пол,Мужчины,НТВ,9.472814,1.136761,717.170027
1,Пол,Женщины,НТВ,9.862545,1.459314,1072.685282
2,Пол,Мужчины,РОССИЯ 1,13.260020,1.591235,1003.892721
3,Пол,Мужчины,ПЕРВЫЙ КАНАЛ,7.557339,0.906899,572.152786
4,Пол,Женщины,РОССИЯ 1,14.849090,2.197149,1615.039635
...,...,...,...,...,...,...
28,Род занятий,Служащий,РОССИЯ 1,10.993879,1.437013,117.059459
29,Род занятий,"Безработный, не работающий",ПЕРВЫЙ КАНАЛ,6.341156,0.906034,44.751648
30,Род занятий,"Пенсионер, инвалид",НТВ,13.653059,3.312643,979.905714
31,Род занятий,"Пенсионер, инвалид",ПЕРВЫЙ КАНАЛ,10.636732,2.580791,763.418267


Настраиваем внешний вид таблицы

In [7]:
# Переименовываем колонку с названием переменной
df.rename(columns={'prj_name':'var_name'}, inplace=True)

df_res = pd.pivot_table(df, values=['Share', 'Rtg000', 'RtgPer'],
                        index=['var_name', 'valueName'], 
                        columns=['tvNetName'])
df_res

Rtg000               \
tvNetName                                                   НТВ ПЕРВЫЙ КАНАЛ   
var_name          valueName                                                    
Возрастная группа 12-17                               17.105817    13.786772   
                  18-24                               25.944214    15.668106   
                  25-34                               75.511878    64.825651   
                  35-44                              140.729911   133.286302   
                  4-11                                22.718649    19.422169   
                  45-54                              254.689960   222.510458   
                  55-64                              466.416027   354.996985   
                  65+                                786.738853   641.667565   
Занятость         Не работает                       1145.959824   890.692609   
                  Работает неполный день              39.171962    31.990622   
                  Работает полный день               604.723522   543.480778   
Образование       высшее                             381.603411   467.461723   
                  начальное                          106.382708    91.277680   
                  среднее                           1301.869189   907.424606   
Пол               Женщины                           1072.685282   894.011223   
                  Мужчины                            717.170027   572.152786   
Род занятий       Безработный, не работающий          59.298532    44.751648   
                  Владелец, совладелец предприятия     8.679115     5.965761   
                  Домохозяйка, молодая мать           39.926602    38.332208   
                  Дошкольник, студент, учащийся       53.813599    40.359201   
                  Другое                              29.158070    18.904890   
                  Индивидуальный бизнес               28.092077    33.908024   
                  Пенсионер, инвалид                 979.905714   763.418267   
                  Рабочий                            213.900491   151.401202   
                  Руководитель                        44.569132    46.317905   
                  Служащий                            92.848771    83.028445   
                  Специалист                         239.663207   239.776458   

                                                                   RtgPer  \
tvNetName                                              РОССИЯ 1       НТВ   
var_name          valueName                                                 
Возрастная группа 12-17                               20.406372  0.187803   
                  18-24                               26.645023  0.278097   
                  25-34                               80.777001  0.368256   
                  35-44                              159.294738  0.625858   
                  4-11                                20.273393  0.156515   
                  45-54                              307.584839  1.382869   
                  55-64                              635.183478  2.370799   
                  65+                               1368.767513  3.487296   
Занятость         Не работает                       1772.688019  1.623436   
                  Работает неполный день              54.985492  1.215564   
                  Работает полный день               791.258846  0.963181   
Образование       высшее                             794.907799  1.124542   
                  начальное                          124.428620  0.365527   
                  среднее                           1699.595938  1.769878   
Пол               Женщины                           1615.039635  1.459314   
                  Мужчины                           1003.892721  1.136761   
Род занятий       Безработный, не работающий          67.045994  1.200548   
                  Владелец, совладелец предприятия     8.105693  1.464034   
                  Домохозяйка, молодая 

## Экспорт в Excel
По умолчанию файл сохраняется в директорию `mediascope-jupyter\mediascope-notebooks\excel`

In [8]:
writer = pd.ExcelWriter(mtask.task_builder.get_excel_filename('01_audience_structure'))
df_info = mtask.task_builder.get_report_info()
df_res.to_excel(writer, 'Report')
df_info.to_excel(writer, 'Info', index=False)
writer.save()